# Elasticsearch | Elasticsearch> [Elasticsearch](https://www.elastic.co/elasticsearch/)は、分散型でRESTfulな検索および分析エンジンであり、ベクトル検索とレキシカル検索の両方を実行できます。これはApache Luceneライブラリの上に構築されています。>> > [Elasticsearch](https://www.elastic.co/elasticsearch/) is a distributed, RESTful search and analytics engine, capable of performing both vector and lexical search. It is built on top of the Apache Lucene library.このノートブックは、`Elasticsearch` データベースに関連する機能の使用方法を示しています。> This notebook shows how to use functionality related to the `Elasticsearch` database.

In [ ]:
!pip install elasticsearch openai tiktoken langchain

## Running and connecting to Elasticsearch | Elasticsearchの実行と接続

Elasticsearchインスタンスを使用するための設定方法は主に2つあります：> There are two main ways to setup an Elasticsearch instance for use with:1. Elastic Cloudは、管理されたElasticsearchサービスです。[無料トライアル](https://cloud.elastic.co/registration?utm_source=langchain\&utm_content=documentation)にご登録ください。   > Elastic Cloud: Elastic Cloud is a managed Elasticsearch service. Signup for a [free trial](https://cloud.elastic.co/registration?utm_source=langchain\&utm_content=documentation).ログイン資格情報が不要なElasticsearchインスタンスに接続するには（セキュリティを有効にしてDockerインスタンスを起動する場合を含む）、ElasticsearchのURLとインデックス名、および埋め込みオブジェクトをコンストラクタに渡してください。> To connect to an Elasticsearch instance that does not require> login credentials (starting the docker instance with security enabled), pass the Elasticsearch URL and index name along with the> embedding object to the constructor.2. ローカルでElasticsearchをインストールする：Elasticsearchをローカルで実行して始めましょう。最も簡単な方法は、公式のElasticsearch Dockerイメージを使用することです。詳細は[Elasticsearch Dockerのドキュメント](https://www.elastic.co/guide/en/elasticsearch/reference/current/docker.html)を参照してください。   > Local Install Elasticsearch: Get started with Elasticsearch by running it locally. The easiest way is to use the official Elasticsearch Docker image. See the [Elasticsearch Docker documentation](https://www.elastic.co/guide/en/elasticsearch/reference/current/docker.html) for more information.### Running Elasticsearch via Docker | Dockerを介してElasticsearchを実行する例：セキュリティを無効にして単一ノードのElasticsearchインスタンスを実行します。これは本番環境での使用には推奨されません。> Example: Run a single-node Elasticsearch instance with security disabled. This is not recommended for production use.```bash    docker run -p 9200:9200 -e "discovery.type=single-node" -e "xpack.security.enabled=false" -e "xpack.security.http.ssl.enabled=false" docker.elastic.co/elasticsearch/elasticsearch:8.9.0```Elasticsearchインスタンスが稼働している場合、ElasticsearchのURLとインデックス名をコンストラクタに指定し、それに埋め込みオブジェクトを使用して接続することができます。> Once the Elasticsearch instance is running, you can connect to it using the Elasticsearch URL and index name along with the embedding object to the constructor.例：> Example:```python        from langchain.vectorstores.elasticsearch import ElasticsearchStore        from langchain.embeddings.openai import OpenAIEmbeddings        embedding = OpenAIEmbeddings()        elastic_vector_search = ElasticsearchStore(            es_url="http://localhost:9200",            index_name="test_index",            embedding=embedding        )```### Authentication | 認証本番環境では、セキュリティを有効にして実行することをお勧めします。ログイン資格情報を使用して接続する場合、`api_key`または`es_user`と`es_password`のパラメータを使用することができます。> For production, we recommend you run with security enabled. To connect with login credentials, you can use the parameters `api_key` or `es_user` and `es_password`.例：> Example:```python        from langchain.vectorstores import ElasticsearchStore        from langchain.embeddings import OpenAIEmbeddings        embedding = OpenAIEmbeddings()        elastic_vector_search = ElasticsearchStore(            es_url="http://localhost:9200",            index_name="test_index",            embedding=embedding,            es_user="elastic",            es_password="changeme"        )```#### How to obtain a password for the default "elastic" user? | デフォルトの「elastic」ユーザーのパスワードを取得する方法は？デフォルトの「elastic」ユーザー用のElastic Cloudパスワードを取得するには：> To obtain your Elastic Cloud password for the default "elastic" user:1. https://cloud.elastic.co でElastic Cloudコンソールにログインしてください   > Log in to the Elastic Cloud console at https://cloud.elastic.co2. 「セキュリティ」>「ユーザー」へ移動してください   > Go to "Security" > "Users"3. 「elastic」ユーザーを見つけて、「編集」をクリックしてください   > Locate the "elastic" user and click "Edit"4. 「パスワードをリセット」をクリックしてください   > Click "Reset password"5. 指示に従ってパスワードをリセットしてください   > Follow the prompts to reset the password#### How to obtain an API key? | APIキーを取得する方法は？APIキーを取得するには：> To obtain an API key:1. https://cloud.elastic.co にアクセスし、Elastic Cloudコンソールにログインしてください   > Log in to the Elastic Cloud console at https://cloud.elastic.co2. Kibanaを開いて、スタック管理 > APIキーに移動してください   > Open Kibana and go to Stack Management > API Keys3. 「APIキーを作成」をクリックしてください   > Click "Create API key"4. APIキーの名前を入力して、「作成」をクリックしてください   > Enter a name for the API key and click "Create"5. APIキーをコピーして、`api_key`パラメータに貼り付けてください   > Copy the API key and paste it into the `api_key` parameter### Elastic Cloud | Elastic CloudElastic Cloud上のElasticsearchインスタンスに接続するには、`es_cloud_id`パラメータまたは`es_url`を使用できます。> To connect to an Elasticsearch instance on Elastic Cloud, you can use either the `es_cloud_id` parameter or `es_url`.例：> Example:```python        from langchain.vectorstores.elasticsearch import ElasticsearchStore        from langchain.embeddings import OpenAIEmbeddings        embedding = OpenAIEmbeddings()        elastic_vector_search = ElasticsearchStore(            es_cloud_id="<cloud_id>",            index_name="test_index",            embedding=embedding,            es_user="elastic",            es_password="changeme"        )```

`OpenAIEmbeddings`を使用するためには、OpenAIのAPIキーを取得する必要があります。> We want to use `OpenAIEmbeddings` so we have to get the OpenAI API Key.

In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Basic Example | 基本的な例この例では、「state\_of\_the\_union.txt」をTextLoaderを通じて読み込み、テキストを500語のチャンクに分割してから、それぞれのチャンクをElasticsearchにインデックス化します。> This example we are going to load "state\_of\_the\_union.txt" via the TextLoader, chunk the text into 500 word chunks, and then index each chunk into Elasticsearch.データがインデックスされた後、私たちは「ケタンジ・ブラウン・ジャクソンについて大統領は何を言ったか」という問いに類似した上位4つのチャンクを見つけるために簡単なクエリを実行します。> Once the data is indexed, we perform a simple query to find the top 4 chunks that similar to the query "What did the president say about Ketanji Brown Jackson".Elasticsearchは、[docker](#running-elasticsearch-via-docker)を介してlocalhost:9200でローカルに実行されています。Elastic CloudからElasticsearchに接続する方法の詳細については、上記の[認証付きで接続する](#authentication)を参照してください。> Elasticsearch is running locally on localhost:9200 with [docker](#running-elasticsearch-via-docker). For more details on how to connect to Elasticsearch from Elastic Cloud, see [connecting with authentication](#authentication) above.

In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import ElasticsearchStore

In [3]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

loader = TextLoader("../../modules/state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

In [4]:
db = ElasticsearchStore.from_documents(
    docs,
    embeddings,
    es_url="http://localhost:9200",
    index_name="test-basic",
)

db.client.indices.refresh(index="test-basic")

query = "What did the president say about Ketanji Brown Jackson"
results = db.similarity_search(query)
print(results)

[Document(page_content='One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.', metadata={'source': '../../modules/state_of_the_union.txt'}), Document(page_content='As I said last year, especially to our younger transgender Americans, I will always have your back as your President, so you can be yourself and reach your God-given potential. \n\nWhile it often appears that we never agree, that isn’t true. I signed 80 bipartisan bills into law last year. From preventing government shutdowns to protecting Asian-Americans from still-too-common hate crimes to reforming military justice.', metadata={'source': '../../modules/state_of_the_union.txt'}), Document(page_content='A former top litigator in private practice. 

# Metadata | メタデータ`ElasticsearchStore`は、ドキュメントと一緒にメタデータを保存する機能をサポートしています。このメタデータ辞書オブジェクトは、Elasticsearchドキュメント内のメタデータオブジェクトフィールドに保存されます。メタデータの値に基づいて、Elasticsearchはメタデータの値のデータタイプを推測して、自動的にマッピングを設定します。例えば、メタデータの値が文字列であれば、Elasticsearchはメタデータオブジェクトフィールドのマッピングを文字列タイプとして設定します。> `ElasticsearchStore` supports metadata to stored along with the document. This metadata dict object is stored in a metadata object field in the Elasticsearch document. Based on the metadata value, Elasticsearch will automatically setup the mapping by infering the data type of the metadata value. For example, if the metadata value is a string, Elasticsearch will setup the mapping for the metadata object field as a string type.

In [5]:
# Adding metadata to documents
for i, doc in enumerate(docs):
    doc.metadata["date"] = f"{range(2010, 2020)[i % 10]}-01-01"
    doc.metadata["rating"] = range(1, 6)[i % 5]
    doc.metadata["author"] = ["John Doe", "Jane Doe"][i % 2]

db = ElasticsearchStore.from_documents(
    docs, embeddings, es_url="http://localhost:9200", index_name="test-metadata"
)

query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)
print(docs[0].metadata)

{'source': '../../modules/state_of_the_union.txt', 'date': '2016-01-01', 'rating': 2, 'author': 'John Doe'}


## Filtering Metadata | メタデータのフィルタリングドキュメントにメタデータを追加することで、クエリ時にメタデータフィルタリングを行うことができます。> With metadata added to the documents, you can add metadata filtering at query time.### Example: Filter by Exact keyword | 例：正確なキーワードによるフィルター注意：分析されないキーワードのサブフィールドを使用しています> Notice: We are using the keyword subfield thats not analyzed

In [6]:
docs = db.similarity_search(
    query, filter=[{"term": {"metadata.author.keyword": "John Doe"}}]
)
print(docs[0].metadata)

{'source': '../../modules/state_of_the_union.txt', 'date': '2016-01-01', 'rating': 2, 'author': 'John Doe'}


### Example: Filter by Partial Match | 例：部分一致によるフィルタリングこの例では、部分一致を用いてフィルタリングする方法を示しています。これは、メタデータフィールドの正確な値がわからない場合に便利です。例えば、メタデータフィールド`author`によってフィルタリングしたいけれども、著者の正確な値がわからないとき、著者の姓を使って部分一致でフィルタリングすることができます。また、あいまい検索もサポートされています。> This example shows how to filter by partial match. This is useful when you don't know the exact value of the metadata field. For example, if you want to filter by the metadata field `author` and you don't know the exact value of the author, you can use a partial match to filter by the author's last name. Fuzzy matching is also supported.「Jon」は「John Doe」に近い一致を示します。というのも、「Jon」は「John」という単語に非常に近い形で一致するためです。> "Jon" matches on "John Doe" as "Jon" is a close match to "John" token.

In [8]:
docs = db.similarity_search(
    query,
    filter=[{"match": {"metadata.author": {"query": "Jon", "fuzziness": "AUTO"}}}],
)
print(docs[0].metadata)

{'source': '../../modules/state_of_the_union.txt', 'date': '2016-01-01', 'rating': 2, 'author': 'John Doe'}


### Example: Filter by Date Range | 例：日付範囲によるフィルター

In [59]:
docs = db.similarity_search(
    "Any mention about Fred?",
    filter=[{"range": {"metadata.date": {"gte": "2010-01-01"}}}],
)
print(docs[0].metadata)

{'source': '../../modules/state_of_the_union.txt', 'date': '2012-01-01', 'rating': 3, 'author': 'John Doe', 'geo_location': {'lat': 40.12, 'lon': -71.34}}


### Example: Filter by Numeric Range | 例：数値範囲でのフィルタリング

In [60]:
docs = db.similarity_search(
    "Any mention about Fred?", filter=[{"range": {"metadata.rating": {"gte": 2}}}]
)
print(docs[0].metadata)

{'source': '../../modules/state_of_the_union.txt', 'date': '2012-01-01', 'rating': 3, 'author': 'John Doe', 'geo_location': {'lat': 40.12, 'lon': -71.34}}


### Example: Filter by Geo Distance | 例：地理的距離によるフィルタリング`metadata.geo_location`にはgeo\_pointマッピングを宣言したインデックスが必要です。> Requires an index with a geo\_point mapping to be declared for `metadata.geo_location`.

In [ ]:
docs = db.similarity_search(
    "Any mention about Fred?",
    filter=[
        {
            "geo_distance": {
                "distance": "200km",
                "metadata.geo_location": {"lat": 40, "lon": -70},
            }
        }
    ],
)
print(docs[0].metadata)

Filterは上記のものよりもさらに多くの種類のクエリをサポートしています。> Filter supports many more types of queries than above.それらについての詳細は、[ドキュメント](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl.html)で読むことができます。> Read more about them in the [documentation](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl.html).

# Distance Similarity Algorithm | 距離類似度アルゴリズムElasticsearchは以下のベクトル距離類似度アルゴリズムをサポートしています：> Elasticsearch supports the following vector distance similarity algorithms:* コサイン  > cosine* ユークリッド  > euclidean* ドット積  > dot\_productコサイン類似度アルゴリズムがデフォルトです。> The cosine similarity algorithm is the default.similarityパラメータを通じて、必要な類似性アルゴリズムを指定することができます。> You can specify the similarity Algorithm needed via the similarity parameter.**注意**検索戦略によっては、クエリ時に類似度アルゴリズムを変更することはできません。これは、フィールドのインデックスマッピングを作成する際に設定する必要があります。類似度アルゴリズムを変更する必要がある場合は、インデックスを削除し、正しい `distance_strategy` を用いて再作成する必要があります。> **NOTE**> Depending on the retrieval strategy, the similarity algorithm cannot be changed at query time. It is needed to be set when creating the index mapping for field. If you need to change the similarity algorithm, you need to delete the index and recreate it with the correct distance\_strategy.```pythondb = ElasticsearchStore.from_documents(    docs,     embeddings,     es_url="http://localhost:9200",     index_name="test",    distance_strategy="COSINE"    # distance_strategy="EUCLIDEAN_DISTANCE"    # distance_strategy="DOT_PRODUCT")```

# Retrieval Strategies | 検索戦略Elasticsearchは、幅広い検索戦略をサポートする能力によって、他のベクター専用データベースよりも大きな利点を有しています。このノートブックでは、いくつかの最も一般的な検索戦略をサポートするように`ElasticsearchStore`を設定する方法を説明します。> Elasticsearch has big advantages over other vector only databases from its ability to support a wide range of retrieval strategies.  In this notebook we will configure `ElasticsearchStore` to support some of the most common retrieval strategies.デフォルトでは、`ElasticsearchStore`は`ApproxRetrievalStrategy`を使用します。> By default, `ElasticsearchStore` uses the `ApproxRetrievalStrategy`.## ApproxRetrievalStrategy | ApproxRetrievalStrategyこれにより、クエリベクトルに最も類似した上位 `k` 個のベクトルが返されます。`k` パラメータは、`ElasticsearchStore` が初期化されるときに設定されます。デフォルト値は `10` です。> This will return the top `k` most similar vectors to the query vector.  The `k` parameter is set when the `ElasticsearchStore` is initialized.  The default value is `10`.

In [ ]:
db = ElasticsearchStore.from_documents(
    docs,
    embeddings,
    es_url="http://localhost:9200",
    index_name="test",
    strategy=ElasticsearchStore.ApproxRetrievalStrategy(),
)

docs = db.similarity_search(
    query="What did the president say about Ketanji Brown Jackson?", k=10
)

### Example: Approx with hybrid | 例：ハイブリッドによる近似この例では、近似意味検索とキーワードベースの検索の組み合わせを使用して、`ElasticsearchStore`を設定しハイブリッド検索を実行する方法を示します。> This example will show how to configure `ElasticsearchStore` to perform a hybrid retrieval, using a combination of approximate semantic search and keyword based search.異なる検索方法から得られた2つのスコアをバランスさせるために、RRFを使用します。> We use RRF to balance the two scores from different retrieval methods.ハイブリッド検索を有効にするためには、`ElasticsearchStore` の `ApproxRetrievalStrategy` コンストラクタで `hybrid=True` を設定する必要があります。> To enable hybrid retrieval, we need to set `hybrid=True` in `ElasticsearchStore` `ApproxRetrievalStrategy` constructor.```pythondb = ElasticsearchStore.from_documents(    docs,     embeddings,     es_url="http://localhost:9200",     index_name="test",    strategy=ElasticsearchStore.ApproxRetrievalStrategy(        hybrid=True,    ))````hybrid`が有効になっている場合、実行されるクエリは、近似的なセマンティック検索とキーワードベースの検索の組み合わせになります。> When `hybrid` is enabled, the query performed will be a combination of approximate semantic search and keyword based search.`rrf`（Reciprocal Rank Fusion）を使用して、異なる検索方法から得られた二つのスコアをバランスさせます。> It will use `rrf` (Reciprocal Rank Fusion) to balance the two scores from different retrieval methods.**注意** RRFはElasticsearch 8.9.0以上が必要です。> **Note** RRF requires Elasticsearch 8.9.0 or above.```json{    "knn": {        "field": "vector",        "filter": [],        "k": 1,        "num_candidates": 50,        "query_vector": [1.0, ..., 0.0],    },    "query": {        "bool": {            "filter": [],            "must": [{"match": {"text": {"query": "foo"}}}],        }    },    "rank": {"rrf": {}},}```### Example: Approx with Embedding Model in Elasticsearch | 例：Elasticsearchにおける埋め込みモデルによる近似この例では、Elasticsearchにデプロイされた埋め込みモデルを使用して近似検索を行うために`ElasticsearchStore`を設定する方法を示します。> This example will show how to configure `ElasticsearchStore` to use the embedding model deployed in Elasticsearch for approximate retrieval.これを使用するには、`ElasticsearchStore` の `ApproxRetrievalStrategy` コンストラクタにおいて `query_model_id` 引数を通じて `model_id` を指定してください。> To use this, specify the model\_id in `ElasticsearchStore` `ApproxRetrievalStrategy` constructor via the `query_model_id` argument.**注意** この機能を使用するには、モデルをElasticsearchのmlノードにデプロイし、実行中である必要があります。モデルをelandを使ってデプロイする方法については、[ノートブックの例](https://github.com/elastic/elasticsearch-labs/blob/main/notebooks/integrations/hugging-face/loading-model-from-hugging-face.ipynb)を参照してください。> **NOTE** This requires the model to be deployed and running in Elasticsearch ml node. See [notebook example](https://github.com/elastic/elasticsearch-labs/blob/main/notebooks/integrations/hugging-face/loading-model-from-hugging-face.ipynb) on how to deploy the model with eland.

In [ ]:
APPROX_SELF_DEPLOYED_INDEX_NAME = "test-approx-self-deployed"

# Note: This does not have an embedding function specified
# Instead, we will use the embedding model deployed in Elasticsearch
db = ElasticsearchStore(
    es_cloud_id="<your cloud id>",
    es_user="elastic",
    es_password="<your password>",
    index_name=APPROX_SELF_DEPLOYED_INDEX_NAME,
    query_field="text_field",
    vector_query_field="vector_query_field.predicted_value",
    strategy=ElasticsearchStore.ApproxRetrievalStrategy(
        query_model_id="sentence-transformers__all-minilm-l6-v2"
    ),
)

# Setup a Ingest Pipeline to perform the embedding
# of the text field
db.client.ingest.put_pipeline(
    id="test_pipeline",
    processors=[
        {
            "inference": {
                "model_id": "sentence-transformers__all-minilm-l6-v2",
                "field_map": {"query_field": "text_field"},
                "target_field": "vector_query_field",
            }
        }
    ],
)

# creating a new index with the pipeline,
# not relying on langchain to create the index
db.client.indices.create(
    index=APPROX_SELF_DEPLOYED_INDEX_NAME,
    mappings={
        "properties": {
            "text_field": {"type": "text"},
            "vector_query_field": {
                "properties": {
                    "predicted_value": {
                        "type": "dense_vector",
                        "dims": 384,
                        "index": True,
                        "similarity": "l2_norm",
                    }
                }
            },
        }
    },
    settings={"index": {"default_pipeline": "test_pipeline"}},
)

db.from_texts(
    ["hello world"],
    es_cloud_id="<cloud id>",
    es_user="elastic",
    es_password="<cloud password>",
    index_name=APPROX_SELF_DEPLOYED_INDEX_NAME,
    query_field="text_field",
    vector_query_field="vector_query_field.predicted_value",
    strategy=ElasticsearchStore.ApproxRetrievalStrategy(
        query_model_id="sentence-transformers__all-minilm-l6-v2"
    ),
)

# Perform search
db.similarity_search("hello world", k=10)

## SparseVectorRetrievalStrategy (ELSER) | SparseVectorRetrievalStrategy（ELSER）この戦略では、Elasticsearchのスパースベクトル検索を使用して、トップkの結果を取得します。現在は、私たち自身の「ELSER」埋め込みモデルのみをサポートしています。> This strategy uses Elasticsearch's sparse vector retrieval to retrieve the top-k results. We only support our own "ELSER" embedding model for now.注意: これを実行するには、ELSERモデルがElasticsearchのMLノードでデプロイされ、実行中である必要があります。> **NOTE** This requires the ELSER model to be deployed and running in Elasticsearch ml node.`ElasticsearchStore` のコンストラクタに `SparseVectorRetrievalStrategy` を指定して利用します。> To use this, specify `SparseVectorRetrievalStrategy` in `ElasticsearchStore` constructor.

In [5]:
# Note that this example doesn't have an embedding function. This is because we infer the tokens at index time and at query time within Elasticsearch.
# This requires the ELSER model to be loaded and running in Elasticsearch.
db = ElasticsearchStore.from_documents(
    docs,
    es_cloud_id="My_deployment:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyQ2OGJhMjhmNDc1M2Y0MWVjYTk2NzI2ZWNkMmE5YzRkNyQ3NWI4ODRjNWQ2OTU0MTYzODFjOTkxNmQ1YzYxMGI1Mw==",
    es_user="elastic",
    es_password="GgUPiWKwEzgHIYdHdgPk1Lwi",
    index_name="test-elser",
    strategy=ElasticsearchStore.SparseVectorRetrievalStrategy(),
)

db.client.indices.refresh(index="test-elser")

results = db.similarity_search(
    "What did the president say about Ketanji Brown Jackson", k=4
)
print(results[0])

page_content='One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.' metadata={'source': '../../modules/state_of_the_union.txt'}


## ExactRetrievalStrategy | ExactRetrievalStrategyこの戦略は、Elasticsearchの厳密な検索（ブルートフォースとも呼ばれる）を利用して、上位k件の結果を取得します。> This strategy uses Elasticsearch's exact retrieval (also known as brute force) to retrieve the top-k results.これを使用するには、`ElasticsearchStore` コンストラクタで `ExactRetrievalStrategy` を指定してください。> To use this, specify `ExactRetrievalStrategy` in `ElasticsearchStore` constructor.```pythondb = ElasticsearchStore.from_documents(    docs,     embeddings,     es_url="http://localhost:9200",     index_name="test",    strategy=ElasticsearchStore.ExactRetrievalStrategy())```

## Customise the Query | クエリのカスタマイズ`custom_query` パラメータを検索に使用することで、Elasticsearchからドキュメントを取得する際に用いるクエリを調整できます。これは、より複雑なクエリを使用したい場合や、フィールドの線形ブースティングをサポートしたい場合に便利です。> With `custom_query` parameter at search, you are able to adjust the query that is used to retrieve documents from Elasticsearch. This is useful if you want to want to use a more complex query, to support linear boosting of fields.

In [11]:
# Example of a custom query thats just doing a BM25 search on the text field.
def custom_query(query_body: dict, query: str):
    """Custom query to be used in Elasticsearch.
    Args:
        query_body (dict): Elasticsearch query body.
        query (str): Query string.
    Returns:
        dict: Elasticsearch query body.
    """
    print("Query Retriever created by the retrieval strategy:")
    print(query_body)
    print()

    new_query_body = {"query": {"match": {"text": query}}}

    print("Query thats actually used in Elasticsearch:")
    print(new_query_body)
    print()

    return new_query_body


results = db.similarity_search(
    "What did the president say about Ketanji Brown Jackson",
    k=4,
    custom_query=custom_query,
)
print("Results:")
print(results[0])

Query Retriever created by the retrieval strategy:
{'query': {'bool': {'must': [{'text_expansion': {'vector.tokens': {'model_id': '.elser_model_1', 'model_text': 'What did the president say about Ketanji Brown Jackson'}}}], 'filter': []}}}

Query thats actually used in Elasticsearch:
{'query': {'match': {'text': 'What did the president say about Ketanji Brown Jackson'}}}

Results:
page_content='One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.' metadata={'source': '../../modules/state_of_the_union.txt'}


# Customize the Document Builder | ドキュメントビルダーのカスタマイズ`search`の`doc_builder`パラメータを使用することで、Elasticsearchから取得したデータを使用してドキュメントがどのように構築されるかを調整できます。これは、Langchainを使用して作成されていないインデックスがある場合に特に便利です。> With `doc_builder` parameter at search, you are able to adjust how a Document is being built using data retrieved from Elasticsearch. This is especially useful if you have indices which were not created using Langchain.

In [ ]:
from typing import Dict

from langchain.docstore.document import Document


def custom_document_builder(hit: Dict) -> Document:
    src = hit.get("_source", {})
    return Document(
        page_content=src.get("content", "Missing content!"),
        metadata={
            "page_number": src.get("page_number", -1),
            "original_filename": src.get("original_filename", "Missing filename!"),
        },
    )


results = db.similarity_search(
    "What did the president say about Ketanji Brown Jackson",
    k=4,
    doc_builder=custom_document_builder,
)
print("Results:")
print(results[0])

# FAQ | よくある質問## Question: Im getting timeout errors when indexing documents into Elasticsearch. How do I fix this? | 質問: Elasticsearchにドキュメントをインデックスする際にタイムアウトエラーが発生しています。これをどのように修正すればいいですか？一つの考えられる問題点は、ドキュメントがElasticsearchにインデックスされるのに時間がかかることです。ElasticsearchStoreはElasticsearchのバルクAPIを使用しており、タイムアウトエラーの可能性を減らすために調整できるいくつかのデフォルト設定があります。> One possible issue is your documents might take longer to index into Elasticsearch. ElasticsearchStore uses the Elasticsearch bulk API which has a few defaults that you can adjust to reduce the chance of timeout errors.これは、SparseVectorRetrievalStrategyを使用しているときにも良いアイデアです。> This is also a good idea when you're using SparseVectorRetrievalStrategy.デフォルト設定は以下の通りです：> The defaults are:* `chunk_size`: 500は、データを処理する際の一回あたりのデータ量を指定するパラメータです。  > `chunk_size`: 500* `max_chunk_bytes`: 100MBは、最大チャンクサイズを100メガバイトに設定します。  > `max_chunk_bytes`: 100MBこれらを調整するには、ElasticsearchStoreの`add_texts`メソッドに`chunk_size`と`max_chunk_bytes`パラメータを渡すことができます。> To adjust these, you can pass in the `chunk_size` and `max_chunk_bytes` parameters to the ElasticsearchStore `add_texts` method.```python    vector_store.add_texts(        texts,        bulk_kwargs={            "chunk_size": 50,            "max_chunk_bytes": 200000000        }    )```

# Upgrading to ElasticsearchStore | ElasticsearchStoreへのアップグレードもし既にlangchainベースのプロジェクトでElasticsearchを使用している場合、`ElasticVectorSearch`や`ElasticKNNSearch`といった古い実装を使用している可能性がありますが、これらは現在非推奨です。より柔軟で使いやすい新しい実装である`ElasticsearchStore`を導入しました。このノートブックは、新しい実装へのアップグレードプロセスを案内します。> If you're already using Elasticsearch in your langchain based project, you may be using the old implementations: `ElasticVectorSearch` and `ElasticKNNSearch` which are now deprecated. We've introduced a new implementation called `ElasticsearchStore` which is more flexible and easier to use. This notebook will guide you through the process of upgrading to the new implementation.## What's new? | 何が新しいですか？新しい実装は、`ElasticsearchStore` という一つのクラスであり、戦略を通じて近似検索、正確検索、およびELSER検索の取得に使用できます。> The new implementation is now one class called `ElasticsearchStore` which can be used for approx, exact, and ELSER search retrieval, via strategies.## Im using ElasticKNNSearch | ElasticKNNSearchを使用しています以前の実装：> Old implementation:```pythonfrom langchain.vectorstores.elastic_vector_search import ElasticKNNSearchdb = ElasticKNNSearch(  elasticsearch_url="http://localhost:9200",  index_name="test_index",  embedding=embedding)```新しい実装：> New implementation:```pythonfrom langchain.vectorstores.elasticsearch import ElasticsearchStoredb = ElasticsearchStore(  es_url="http://localhost:9200",  index_name="test_index",  embedding=embedding,  # if you use the model_id  # strategy=ElasticsearchStore.ApproxRetrievalStrategy( query_model_id="test_model" )  # if you use hybrid search  # strategy=ElasticsearchStore.ApproxRetrievalStrategy( hybrid=True ))```## Im using ElasticVectorSearch | ElasticVectorSearchを使用しています以前の実装：> Old implementation:```pythonfrom langchain.vectorstores.elastic_vector_search import ElasticVectorSearchdb = ElasticVectorSearch(  elasticsearch_url="http://localhost:9200",  index_name="test_index",  embedding=embedding)```新しい実装：> New implementation:```pythonfrom langchain.vectorstores.elasticsearch import ElasticsearchStoredb = ElasticsearchStore(  es_url="http://localhost:9200",  index_name="test_index",  embedding=embedding,  strategy=ElasticsearchStore.ExactRetrievalStrategy())```

In [6]:
db.client.indices.delete(
    index="test-metadata, test-elser, test-basic",
    ignore_unavailable=True,
    allow_no_indices=True,
)

ObjectApiResponse({'acknowledged': True})